## This notebook performs CWFS for a target, but only measures and does not apply the corrections automatically

#### WARNING: This currently requires ts_externalscripts branch **tickets/DM-28818**

In [ ]:
TARGET='HD 85552'

In [ ]:
import os
import sys
import asyncio
import time

import numpy as np
import logging 
import yaml
import matplotlib.pyplot as plt
import astropy

from lsst.ts import salobj
from lsst.ts.externalscripts.auxtel.latiss_cwfs_align import LatissCWFSAlign
from lsst.ts.externalscripts.auxtel.latiss_acquire_and_take_sequence import LatissAcquireAndTakeSequence

from lsst.ts.idl.enums.Script import ScriptState

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
# for autocompleted to work
%config IPCompleter.use_jedi = False

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)
# if you want logging
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

# turn off logging for matplotlib
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.WARNING)

In [ ]:
script_cwfs = LatissCWFSAlign(index=1, remotes=False)  # this essentially calls the init method
await script_cwfs.start_task

In [ ]:
# set wrap strategy
# this is required until the ATPtg is updated to not configure the mount for maximum time on target
script_cwfs.atcs.rem.atptg.cmd_raDecTarget.set(azWrapStrategy=1)  # 1 does not unwrap, 0 unwraps

## ONLY RUN THESE CELLS IF AT THE POLE

In [ ]:
# reset the offsets
await script_cwfs.atcs.rem.ataos.cmd_resetOffset.set_start(axis='x')
await script_cwfs.atcs.rem.ataos.cmd_resetOffset.set_start(axis='y')
await script_cwfs.atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [ ]:
# now add back the values that were used at the pole earlier (which should be the same)
pole_focus= 0.0
pole_coma_x = 0.0
pole_coma_y = 0.0

In [ ]:
await script_cwfs.atcs.rem.ataos.cmd_offset.set_start(z=pole_focus, x=pole_coma_x, y=pole_coma_y)

## Perform CWFS observations

In [ ]:
cwfs_configuration = yaml.safe_dump({"filter": 'RG610', 
                                "grating": 'empty_1',
                                "exposure_time": 20,
                                "dataPath": '/project/shared/auxTel/rerun/quickLook'})
print(cwfs_configuration)

In [ ]:
# Set script state to UNCONFIGURED
# this is required to run the script a 2nd time but otherwise is a no-op
script_cwfs.set_state(ScriptState.UNCONFIGURED)
# Configure the script, which puts the ScriptState to CONFIGURED
cwfs_config_data = script_cwfs.cmd_configure.DataType()
cwfs_config_data.config = cwfs_configuration
await script_cwfs.do_configure(cwfs_config_data)

In [ ]:
# set group ID as before
time_for_groupID=astropy.time.Time.now().isot
group_id_data = script_cwfs.cmd_setGroupId.DataType(
                groupId=time_for_groupID
            )
await script_cwfs.do_setGroupId(group_id_data)

In [ ]:
# Run this script while taking new data
script_cwfs.intra_visit_id = None
script_cwfs.extra_visit_id = None
script_cwfs.short_timeout = 10

In [ ]:
# # run acquisition script
# # results = await script_cwfs.run()
# result = script_cwfs.cmd_run.DataType()
# await script_cwfs.do_run(result)
# await script_cwfs.done_task

In [ ]:
# Run this script while taking new data
# this will print but not apply the corrections
results = await script_cwfs.run_cwfs()

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script.intra_visit_id}")
ax11.imshow(script_cwfs.I1[0].image0)
ax11.contour(script_cwfs.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script.extra_visit_id}")
ax12.imshow(script_cwfs.I2[0].image0)
ax12.contour(script_cwfs.algo.pMask) 

In [ ]:
await atcs.take_object(exptime=30)

In [ ]:
# Show which files/parameters were taken in the sequence above
print(f'intra_visit_id is {script_cwfs.intra_visit_id}')
print(f'extra_visit_id is {script_cwfs.extra_visit_id}')
print(f'angle is {script_cwfs.angle}')

In [ ]:
# Apply calculated focus offset
calculated_hexapod_focus_offset = results['hex_offset'][2]
coma_x = results["hex_offset"][0]
coma_y = results["hex_offset"][1]

print(f'Applying hexapod offsets [x,y,z] of [{coma_x:0.5f}, {coma_y:0.5f}, {calculated_hexapod_focus_offset:0.5f}] mm')
#flush summary event
script_cwfs.atcs.rem.ataos.evt_focusOffsetSummary.flush()
await script_cwfs.atcs.rem.ataos.cmd_offset.set_start(z=calculated_hexapod_focus_offset, x=coma_x, y=coma_y)

In [ ]:
# Apply telescope offsets
tel_x, tel_y = results["tel_offset"][0], -results["tel_offset"][1]
print(f'Applying telescope offsets [x,y] of [{tel_x:0.2f}, {tel_y:0.2f}] arcsec')
await script_cwfs.atcs.offset_xy(x=tel_x, y=tel_y, relative=True, persistent=True)

In [ ]:
# look at current focus offsets (and paste into the log)
focus_summary = await script_cwfs.atcs.rem.ataos.evt_focusOffsetSummary.next(flush=False, timeout=5)

In [ ]:
print(focus_summary)

In [ ]:
# take a quick object image to check PSF centering
await take_object(exptime=5)

## RUN THIS NEXT CELL ONLY IF DATA IS ACCEPTABLE FOR LUT GENERATION

In [ ]:
tmp = await script_cwfs.atcs.rem.athexapod.tel_positionStatus.next(flush=True, timeout=5)
script_cwfs.log.info(f'Hexapod LUT Datapoint - {TARGET} - reported hexapod position is, {tmp.reportedPosition} ')

#### BELOW IS ONLY FOR TESTING PURPOSES

In [ ]:
# If desired then different filenames can be manually input here
script_cwfs.intra_visit_id=2021021800591
script_cwfs.extra_visit_id=2021021800592 
script_cwfs.angle=0

In [ ]:
# reruns reduction part only. ALL 3 fields above must be set! 
rerun_results = await script_cwfs.run_cwfs()

In [ ]:
results=rerun_results

In [ ]:
%matplotlib inline

In [ ]:
# show donuts and centroids
fig1 = plt.figure(1, figsize=(12,8))
ax11 = fig1.add_subplot(121)
ax11.set_title(f"intra visitID - {script_cwfs.intra_visit_id}")
ax11.imshow(script_cwfs.I1[0].image0)
ax11.contour(script_cwfs.algo.pMask) 
ax12 = fig1.add_subplot(122)
ax12.set_title(f"extra visitID - {script_cwfs.extra_visit_id}")
ax12.imshow(script_cwfs.I2[0].image0)
ax12.contour(script_cwfs.algo.pMask) 

In [ ]:
rerun_results.I1